In [56]:
#!/usr/bin/env python

'''
use pymetadata module to create a minimal valid combine archive
using LEMS_NML2_Ex9_FN.sbml and LEMS_NML2_Ex9_FN.sedml
'''

import sys
sys.path.append("..")
import utils
import os
import pandas as pd

sbml_filepath = 'LEMS_NML2_Ex9_FN.sbml'
#sedml_filepath = 'LEMS_NML2_Ex9_FN.sedml' #xmlns:sbml added manually
sedml_filepath = 'LEMS_NML2_Ex9_FN_missing_xmlns.sedml' #xmlns:sbml missing

# pip install biosimulators-gillespy2, biosimulators-copasi, biosimulators-cobrapy, biosimulators-cbmpy, biosimulators-boolnet, biosimulators-bionetgen, 
# pip install biosimulators-pyneuroml[neuron], biosimulators-pyneuroml[brian2], pip install biosimulators-pyneuroml[netpyne]
# 
# docker pull ghcr.io/biosimulators/brian2
# docker pull ghcr.io/biosimulators/netpyne
# docker pull ghcr.io/biosimulators/neuron
# docker pull ghcr.io/biosimulators/pyneuroml

# For OpenCOR:https://docs.biosimulators.org/Biosimulators_OpenCOR/installation.html#python-package-and-command-line-program
# For rbapy: https://docs.biosimulators.org/Biosimulators_RBApy/installation.html#command-line-program
# smoldyn: https://smoldyn.readthedocs.io/en/latest/python/api.html#sed-ml-combine-biosimulators-api
# vcell https://github.com/virtualcell/vcell

# get folder that the current script is in

cwd = os.getcwd()
print(cwd)

c:\Users\prins\GitHub\SBMLShowcase\SBML


In [57]:
engines = ['amici',\
           'brian2',\
           'bionetgen',\
           'boolnet',\
           'cbmpy',\
           'cobrapy',\
           'copasi',\
           'gillespy2',\
           'tellurium',\
           'libsbmlsim',\
           'ginsim',\
           'masspy',\
           'opencor',\
           'pysces',\
           'rbapy',\
           'vcell',\
           'xpp',  
           'smoldyn'  
            ]

engine_dict = {}

for e in engines:
    print('Running ' + e)
    # continue even if error occurs
    output_dir = os.path.abspath(os.path.join('output', e))
    print(output_dir)
    try:
        record = utils.run_biosimulators_docker(e, sedml_filepath, sbml_filepath, output_dir=output_dir)
        engine_dict[e] = record
    except Exception as error:
        # get error message
        error_message = str(error)
        print(f"Error occurred while running {e}")
        engine_dict[e] = error_message
        continue

Running amici
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\amici
Running brian2
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\brian2
Running bionetgen
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\bionetgen
Running boolnet
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\boolnet
Running cbmpy
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\cbmpy
Running cobrapy
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\cobrapy
Running copasi
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\copasi
Running gillespy2copasi
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\gillespy2copasi
Running tellurium
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\tellurium
Running libsbmlsim
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\libsbmlsim
Running ginsim
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\ginsim
Running masspy
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\masspy
Running opencor
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\opencor
Running pysces
c:\Users\prins\GitHub\SBMLShowcase\SBML\output\p

In [58]:
results_table = pd.DataFrame.from_dict(engine_dict, orient='index', columns=['Result'])
results_table.index.name = 'Engine'
results_table.reset_index(inplace=True)

if 'Result' in results_table.columns:
    results_table['pass/Fail'] = results_table['Result'].apply(lambda x: x[0] if isinstance(x, list) else x)
    results_table['Error'] = results_table['Result'].apply(lambda x: x[1] if isinstance(x, list) else None)

results_table.drop(columns=['Result'], inplace=True)
results_table

# write to md file in current directory
results_table.to_markdown('results.md', index=False)

In [59]:
results_table
# save as md file
results_table.to_markdown('results.md', index=False)
# write in cwd
with open('results.md', 'w') as f:
    f.write(results_table.to_markdown(index=False))

In [60]:
df = pd.DataFrame.from_dict(engine_dict)
# turn the dataframe around
df = df.T
df

,0,1
amici,pass,pass
brian2,other,```Command '-i /root/in/LEMS_NML2_Ex9_FN_missi...
bionetgen,other,```Command '-i /root/in/LEMS_NML2_Ex9_FN_missi...
boolnet,other,```Command '-i /root/in/LEMS_NML2_Ex9_FN_missi...
cbmpy,other,```Command '-i /root/in/LEMS_NML2_Ex9_FN_missi...
cobrapy,other,```Command '-i /root/in/LEMS_NML2_Ex9_FN_missi...
copasi,other,```Command '-i /root/in/LEMS_NML2_Ex9_FN_missi...
gillespy2copasi,other,```500 Server Error for http+docker://localnpi...
tellurium,pass,pass
libsbmlsim,pass,pass


In [61]:
results_table = pd.DataFrame()

for k in engine_dict.keys():

    # if engine_dict[k] is a string get string, if list get first item from list
    if type(engine_dict[k]) == str:
        results_table['pass/FAIL'] = [engine_dict[k]]
    else:
        results_table['pass/FAIL'] = [engine_dict[k][0]]
